In [1]:
import keras
import numpy as np

Using TensorFlow backend.
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
path = keras.utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path,encoding='utf-8').read().lower()
print("Corpus length:", len(text))

Corpus length: 600893


#### 将字符序列向量化

In [4]:
maxlen = 30 # 提取60个字符组成的序列
step = 3 # 每3个字符采样一个新序列
sentences = [] # 保存所提取的序列
next_chars = [] # 保存目标

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])
    
print("Number of sequences:", len(sentences))

Number of ssquences: 200288


In [5]:
chars = sorted(list(set(text))) # 预料中唯一字符组成的列表
print("Unique characters:", len(chars))
chars_indices = dict((char, chars.index(char)) for char in chars)

Unique characters: 57


In [6]:
print("Vectorization...")
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i,t,chars_indices[char]] = 1
    y[i, chars_indices[next_chars[i]]] = 1 # 将字符one-hot编码为二进制数组

Vectorization...


#### 构建网络

In [7]:
# 勇于预测下一个字符的单层LSTM模型
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))
# 目标是经过one-hot编码的,索引训练模型需要使用categorical_crossentropy作为损失

In [8]:
# 模型编译配置
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

#### 训练语言模型并从中采样

In [10]:
# 对模型得到的原始概率分布进行重新甲醛, 并从中抽取一个字符索引
def sample(preds, temperature=1.0):
    preds = np.asscalar(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [13]:
# 文本生成循环
import random 
import sys 

for epoch in range(1, 60):
    print("epoch", epoch)
    model.fit(x,y,batch_size=128,epochs=1)
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index:start_index+maxlen]
    print('---Generating with seed:',generated_text) 
    
#     尝试一系列不同的采样更堵
    for tempperature in [0.2, 0.5, 1.0, 1.2]:
        print('-----tempperature',tempperature)
        sys.stdout.write(generated_text)
        
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0,t,chars_indices[char]]=1
                
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, tempperature)
            next_char =chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)

epoch 1
Epoch 1/1
200288/200288 [==============================] - 175s 875us/step - loss: 1.5552
---Generating with seed:  one goes away richer; not fav
-----temperature 0.2
 one goes away richer; not fav

ValueError: can only convert an array of size 1 to a Python scalar